In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 1
%aimport utils, GroupingAlgorithm

In [2]:
import numpy as np
import networkx as nx
import warnings

from qiskit import IBMQ

from utils import molecules, Label2Chain, get_backend_connectivity, number_cnots_raw
from GroupingAlgorithm import grouping, groupingWithOrder

In [3]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic')

In [4]:
name_backend = 'ibmq_montreal'
backend_device = provider.get_backend(name_backend)
WC_device = get_backend_connectivity(backend_device)
G_device = nx.Graph()
G_device.add_edges_from(WC_device)

In [ ]:
molecule_names = ['H2', 'LiH', 'BeH2', 'H2O', 'CH4', 'C2H2', 'CH3OH', 'C2H6']
qubit_ops = []
paulis = []
coeffs = []
labels = []
num_qubits = []

for molecule in molecule_names:
    print('Molecule: {}'.format(molecule))
    try:
        qubit_ops.append(np.load('../data/big_molecules.npy', allow_pickle=True).item()[molecule])
        print('  Data loaded')
    except KeyError:
        print('  Computing qubit_operator ...')
        qubit_ops.append(molecules(molecule))
    
    temp = Label2Chain(qubit_ops[-1])
    paulis.append(temp[0])
    coeffs.append(temp[1])
    labels.append(temp[2])
    
    num_qubits.append(qubit_ops[-1].num_qubits)
    
    print('-' * 30)

# EM

In [ ]:
Groups_EM = []
Measurements_EM = []
n_cnots_EM = []

for i in range(len(molecule_names)):    
    temp = grouping(paulis[i])
    Groups_EM.append(temp[0])
    Measurements_EM.append(temp[1])
    
    n_cnots_EM.append(number_cnots_raw(Measurements_EM[i], num_qubits[i], WC_device))
    
    print('{}: {} groups with {} cnots'.format(molecule_names[i], len(Groups_EM[i]), n_cnots_EM[i]))

# HEEM (all)

In [ ]:
Groups_HEEM_all = []
Measurements_HEEM_all = []
n_cnots_HEEM_all = []

for i in range(len(molecule_names)):
    temp = groupingWithOrder(paulis[i], connected=True)
    Groups_HEEM_all.append(temp[0])
    Measurements_HEEM_all.append(temp[1])
    
    n_cnots_HEEM_all.append(number_cnots_raw(Measurements_HEEM_all[i], num_qubits[i], WC_device))
    
    print('{}: {} groups with {} cnots'.format(molecule_names[i], len(Groups_HEEM_all[i]), n_cnots_HEEM_all[i]))

# HEEM (montreal)

In [ ]:
Groups_HEEM_mont = []
Measurements_HEEM_mont = []
n_cnots_HEEM_mont = []

for i in range(len(molecule_names)):
    temp = groupingWithOrder(paulis[i], G_device, connected=True)
    Groups_HEEM_mont.append(temp[0])
    Measurements_HEEM_mont.append(temp[1])
    T = temp[2]
    
    n_cnots_HEEM_mont.append(number_cnots_raw(Measurements_HEEM_mont[i], num_qubits[i], WC_device, T))
    
    print('{}: {} groups with {} cnots'.format(molecule_names[i], len(Groups_HEEM_mont[i]), n_cnots_HEEM_mont[i]))

# Results

## EM
| Molecule                    | Groups      | # CNOTS     |
| :------:                    | :----:      | :------:    |
| H<sub>2</sub>               | 2           | 1           |
| LiH                         | 11          | 9           |
| BeH<sub>2</sub>             | 15          | 30          |
| H<sub>2</sub>O              | 51          | 152         |
| CH<sub>4</sub>              | 113         | 426         |
| C<sub>2</sub>H<sub>2</sub>  | 189         | 1152        |
| CH<sub>3</sub>OH            | 688         | 6066        |
| C<sub>2</sub>H<sub>6</sub>  | 767         | 8104        |

## HEEM (all)
| Molecule                    | Groups      | # CNOTS     |
| :------:                    | :----:      | :------:    |
| H<sub>2</sub>               | 2           | 1           |
| LiH                         | 11          | 9           |
| BeH<sub>2</sub>             | 13          | 20          |
| H<sub>2</sub>O              | 48          | 99          |
| CH<sub>4</sub>              | 115         | 412         |
| C<sub>2</sub>H<sub>2</sub>  | 205         | 1277        |
| CH<sub>3</sub>OH            | 682         | 6082        |
| C<sub>2</sub>H<sub>6</sub>  | 758         | 7994        |


## HEEM (all)
| Molecule                    | Groups      | # CNOTS     |
| :------:                    | :----:      | :------:    |
| H<sub>2</sub>               | 2           | 1           |
| LiH                         | 11          | 9           |
| BeH<sub>2</sub>             | 19          | 25          |
| H<sub>2</sub>O              | 53          | 61          |
| CH<sub>4</sub>              | 169         | 246         |
| C<sub>2</sub>H<sub>2</sub>  | 287         | 433         |
| CH<sub>3</sub>OH            | 1503        | 2770        |
| C<sub>2</sub>H<sub>6</sub>  | 1529        | 2873        |